In [1]:
import ipywidgets as widgets
from IPython.display import display
from queue import Queue
from threading import Thread

messages = Queue() #tell the thread when to stop recording
recordings = Queue() #store the audio from the mic. and pass to transcription

record_button = widgets.Button(
    description = "Record",
    disabled = False,
    button_style = "success",
    icon = "microphone"
)

stop_button = widgets.Button(
    description = "Stop",
    disabled = False,
    button_style = "warning",
    icon = "stop"
)

output = widgets.Output()

def start_recording(data):
    messages.put(True) #keep running & recording the mic.

    #put a message onto the messages queue which tells the threads to keep running
    with output:
        display("Starting...")
        record = Thread(target = record_mic) #calls a thread that will record the mic
        record.start() #start thread and have it record in the background
        
        transcribe = Thread(target=speech_recognition, args=(output,))
        transcribe.start() #transcribe audio into text
    

def stop_recording(data):
    with output:
        messages.get() #takes the message off the queue
        display("Stopped.")

record_button.on_click(start_recording)
stop_button.on_click(stop_recording)

display(record_button, stop_button) 

Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle())

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle())

In [ ]:
import pyaudio #microphone index = 0
p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))
    
p.terminate()

In [ ]:
CHANNELS = 1
FRAME_RATE = 16000
RECORD_SECONDS = 20
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2

def record_mic(chunk=1024):
    p = pyaudio.PyAudio()
    
    stream = p.open(format=AUDIO_FORMAT,
                    channels=CHANNELS,
                    rate=FRAME_RATE,
                    input=True,
                    input_device_index=2,
                    frames_per_buffer=chunk)
    frames = []
    while not messages.empty():
        data = stream.read(chunk)
        frames.append(data)
        
        if len(frames) >= (FRAME_RATE * RECORD_SECONDS) / chunk:
            recordings.put(frames.copy())
            frames
    